# Covid-19 (Coronavirus): Data Extract, Transform, Load (ETL) Notebook
This notebook pulls data from various sources, performs data wrangling to get it into more usable formats, and exports to CSV for use in other notebooks, as well as for future reference.  

Feeds into the Covid19_Analytics notebook.

## Data Sources

### Primary Data Source: Johns Hopkins CSSE Data Repository 
- Link: https://github.com/CSSEGISandData/COVID-19  
- **Live data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US County (cases, deaths)
- **Historic time series data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US Counties (cases, deaths)

### US State Testing and Hospitalizations: Covid Tracking Project 
- Link: https://covidtracking.com/data/ 
- **Live data:**
    - US State testing and hospitalization, ICU stats
- **Historic time series data:**
    - US State testing and hospitalization, ICU stats

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime
import os

## Get the Datasets

### Dataset 1: John Hopkins

Live data is stored in files named MM-DD-YYYY.csv, so get the current date for live data updates.  
https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports

#### 1A: Pull the Data from JH's github repo, immediately export as 'orig'

In [2]:
# Create the filename to load based on today's date

day = str(datetime.now().day) if datetime.now().day >= 10 else "0" + str(datetime.now().day)
month = str(datetime.now().month) if datetime.now().month >= 10 else "0" + str(datetime.now().month)
year = str(datetime.now().year)

jh_live_date = month + "-" + day + "-" + year
jh_live_date

'04-22-2021'

In [3]:
#  First try to pull today's data, but if it's not available yet, pull yesterday's data
try:
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
except:
    day = str(datetime.now().day-1) if datetime.now().day-1 >= 10 else "0" + str(datetime.now().day-1)
    jh_live_date = month + "-" + day + "-" + year
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
    
jh_live_date

'04-22-2021'

In [4]:
# Check out the first x rows to make sure the data loaded correctly
jh_live_global.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
0,NaN,NaN,NaN,Afghanistan,2021-04-23 04:20:52,33.93911,67.709953,58312,2561,52348.0,3403.0,Afghanistan,149.793180,4.391892
1,NaN,NaN,NaN,Albania,2021-04-23 04:20:52,41.15330,20.168300,130114,2364,103582.0,24168.0,Albania,4521.300994,1.816868
2,NaN,NaN,NaN,Algeria,2021-04-23 04:20:52,28.03390,1.659600,120363,3181,83900.0,33282.0,Algeria,274.481499,2.642839
3,NaN,NaN,NaN,Andorra,2021-04-23 04:20:52,42.50630,1.521800,12942,123,12375.0,444.0,Andorra,16750.145603,0.950394
4,NaN,NaN,NaN,Angola,2021-04-23 04:20:52,-11.20270,17.873900,25051,572,22901.0,1578.0,Angola,76.221006,2.283342


Now immediately export the original jh dataset before making any changes

In [5]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_live_global_orig.csv')
jh_live_global.to_csv(filename, index=False)

**Also pull the global historic time series data.  Note that it won't have live data from today until close to midnight PT.**

In [6]:
jh_hist_global_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
jh_hist_global_cases.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,4/13/21,4/14/21,4/15/21,4/16/21,4/17/21,4/18/21,4/19/21,4/20/21,4/21/21,4/22/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,57364,57492,57534,57612,57721,57793,57898,58037,58214,58312
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,128752,128959,129128,129307,129456,129594,129694,129842,129980,130114
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,118799,118975,119142,119323,119486,119642,119805,119992,120174,120363


In [7]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_cases_orig.csv')
jh_hist_global_cases.to_csv(filename, index=False)

In [8]:
jh_hist_global_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
jh_hist_global_deaths.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,4/13/21,4/14/21,4/15/21,4/16/21,4/17/21,4/18/21,4/19/21,4/20/21,4/21/21,4/22/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,2529,2532,2533,2535,2539,2539,2546,2549,2557,2561
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,2326,2331,2335,2337,2340,2342,2347,2353,2358,2364
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,3137,3141,3144,3148,3152,3155,3160,3165,3172,3181


In [9]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_deaths_orig.csv')
jh_hist_global_deaths.to_csv(filename, index=False)

In [10]:
jh_hist_global_recovered = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
jh_hist_global_recovered.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,4/13/21,4/14/21,4/15/21,4/16/21,4/17/21,4/18/21,4/19/21,4/20/21,4/21/21,4/22/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,52013,52022,52083,52105,52116,52168,52244,52272,52301,52348
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,98903,99441,100013,100600,101142,101584,102171,102601,103066,103582
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,82813,82929,83048,83169,83286,83397,83514,83636,83765,83900


In [11]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_recovered_orig.csv')
jh_hist_global_recovered.to_csv(filename, index=False)

In [12]:
jh_hist_uscounties_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')
jh_hist_uscounties_cases.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,4/13/21,4/14/21,4/15/21,4/16/21,4/17/21,4/18/21,4/19/21,4/20/21,4/21/21,4/22/21
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,6734,6740,6748,6750,6760,6763,6763,6773,6793,6819
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,20652,20670,20674,20701,20714,20723,20730,20764,20787,20815
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,2252,2257,2262,2264,2271,2271,2271,2275,2284,2289


In [13]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_cases_orig.csv')
jh_hist_uscounties_cases.to_csv(filename, index=False)

In [14]:
jh_hist_uscounties_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')
jh_hist_uscounties_deaths.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,4/13/21,4/14/21,4/15/21,4/16/21,4/17/21,4/18/21,4/19/21,4/20/21,4/21/21,4/22/21
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,103,103,103,103,106,106,106,106,107,107
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,300,300,302,302,302,302,302,302,302,303
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,55,55,55,55,55,55,55,55,55,56


In [15]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_deaths_orig.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=False)

#### 1B: Wrangle the data into more usable format

For the JH live global data, it will be useful to group / subtotal by Country, by US State, and by US County.  

For the historic time series data, these need to be rearranged so that the date is in 
datetime format and pivoted so the date is the index of the DataFrame.

In [16]:
jh_live_global[jh_live_global['Country_Region']=='US'].sort_values('Province_State').head(3)

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
652,1001.0,Autauga,Alabama,US,2021-04-23 04:20:52,32.539527,-86.644082,6819,107,NaN,NaN,"Autauga, Alabama, US",12205.337486,1.569145
689,1075.0,Lamar,Alabama,US,2021-04-23 04:20:52,33.779950,-88.096680,1398,34,NaN,NaN,"Lamar, Alabama, US",10126.765665,2.432046
690,1077.0,Lauderdale,Alabama,US,2021-04-23 04:20:52,34.901719,-87.656247,9301,235,NaN,NaN,"Lauderdale, Alabama, US",10030.303357,2.526610


##### 1B-1: Pivot Table on the JH live global data, grouped / subtotalled by **Country**

In [17]:
jh_live_countries = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Confirmed', 'Deaths', 'Recovered'], aggfunc=np.sum)

In [18]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Country_Region,,,
US,31928507,570321,0.0
India,16263695,186920,13648159.0
Brazil,14167973,383502,12530970.0


In [19]:
jh_live_countries['Active'] = jh_live_countries['Confirmed'] - jh_live_countries['Deaths'] - jh_live_countries['Recovered']

In [20]:
jh_live_countries['Last_Update'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Last_Update'], aggfunc=np.max)
jh_live_countries['Lat'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Lat'], aggfunc=np.mean)
jh_live_countries['Long'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Long_'], aggfunc=np.mean)

In [21]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Country_Region,,,,,,,
US,31928507,570321,0.0,31358186.0,2021-04-23 04:20:52,37.950547,-91.419547
India,16263695,186920,13648159.0,2428616.0,2021-04-23 04:20:52,23.088275,81.806127
Brazil,14167973,383502,12530970.0,1253501.0,2021-04-23 04:20:52,-12.669522,-48.480493
France,5469674,102323,334497.0,5032854.0,2021-04-23 04:20:52,6.430808,-34.730285
Russia,4682573,105328,4312186.0,265059.0,2021-04-23 04:20:52,54.546312,62.120860


##### 1B-2: Pivot Table on the JH live global data, grouped / subtotalled by **US State**

In [22]:
jh_live_usstates = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [23]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Province_State,,,
California,3727069,61285,0.0
Texas,2867748,49818,0.0
Florida,2191038,34696,0.0


In [24]:
jh_live_usstates['Active'] = jh_live_usstates['Confirmed'] - jh_live_usstates['Deaths'] - jh_live_usstates['Recovered']

In [25]:
jh_live_usstates['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_usstates['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_usstates['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Long_'], 
                                    aggfunc=np.mean)

In [26]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Province_State,,,,,,,
California,3727069,61285,0.0,3665784.0,2021-04-23 04:20:52,37.843962,-120.728594
Texas,2867748,49818,0.0,2817930.0,2021-04-23 04:20:52,31.660643,-98.653069
Florida,2191038,34696,0.0,2156342.0,2021-04-23 04:20:52,28.940755,-82.700744
New York,2018044,51830,0.0,1966214.0,2021-04-23 04:20:52,42.544151,-75.474183
Illinois,1312620,24056,0.0,1288564.0,2021-04-23 04:20:52,39.843618,-89.178330


##### 1B-3: Pivot Table on the JH live global data, grouped / subtotalled by **US County**

In [27]:
jh_live_uscounties = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [28]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered
Province_State,Admin2,,,
California,Los Angeles,1230362,23736,0.0
Arizona,Maricopa,532809,9803,0.0
Illinois,Cook,525854,9860,0.0
Florida,Miami-Dade,475231,6077,0.0
Texas,Harris,388423,6122,0.0


In [29]:
jh_live_uscounties['Active'] = jh_live_uscounties['Confirmed'] - jh_live_uscounties['Deaths'] - jh_live_uscounties['Recovered']


In [30]:
jh_live_uscounties['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_uscounties['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_uscounties['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Long_'], 
                                    aggfunc=np.mean)
jh_live_uscounties['FIPS'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['FIPS'], 
                                    aggfunc=np.max)

In [31]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long,FIPS
Province_State,Admin2,,,,,,,,
California,Los Angeles,1230362,23736,0.0,1206626.0,2021-04-23 04:20:52,34.308284,-118.228241,6037.0
Arizona,Maricopa,532809,9803,0.0,523006.0,2021-04-23 04:20:52,33.348359,-112.491815,4013.0
Illinois,Cook,525854,9860,0.0,515994.0,2021-04-23 04:20:52,41.841448,-87.816588,17031.0
Florida,Miami-Dade,475231,6077,0.0,469154.0,2021-04-23 04:20:52,25.611236,-80.551706,12086.0
Texas,Harris,388423,6122,0.0,382301.0,2021-04-23 04:20:52,29.858649,-95.393395,48201.0


##### 1B-4: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [32]:
# First, isolate the date columns of the JH historic time series data to be used in the following 3 Pivot Tables
date_columns = jh_hist_global_cases.iloc[:, 4:].columns

In [33]:
jh_hist_countries_cases = pd.pivot_table(jh_hist_global_cases, index=['Country/Region'], 
                                            values=date_columns, aggfunc=np.sum)
jh_hist_countries_cases = jh_hist_countries_cases.transpose()
jh_hist_countries_cases.index = pd.to_datetime(jh_hist_countries_cases.index)
jh_hist_countries_cases = jh_hist_countries_cases.sort_index()

In [34]:
jh_hist_countries_cases.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-04-20,58037,129842,119992,12874,24661,1217,2743620,209485,29576,597566,...,4408644,169327,87225,3,185736,2800,284280,5918,91042,37875
2021-04-21,58214,129980,120174,12917,24883,1217,2769552,210518,29594,600089,...,4411068,172601,87551,4,186745,2812,286028,5960,91119,37980
2021-04-22,58312,130114,120363,12942,25051,1217,2796768,211399,29638,602494,...,4413834,175891,87935,4,188063,2824,287680,6020,91189,38018


##### 1B-5: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **Country**

In [35]:
jh_hist_countries_deaths = pd.pivot_table(jh_hist_global_deaths, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_deaths = jh_hist_countries_deaths.transpose()
jh_hist_countries_deaths.index = pd.to_datetime(jh_hist_countries_deaths.index)
jh_hist_countries_deaths = jh_hist_countries_deaths.sort_index()

In [36]:
jh_hist_countries_deaths.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-04-20,2549,2353,3165,123,565,31,59792,3919,910,9959,...,127557,2022,639,0,1944,35,3078,1138,1236,1554
2021-04-21,2557,2358,3172,123,570,31,60083,3944,910,9997,...,127577,2083,640,1,1965,35,3096,1147,1238,1555
2021-04-22,2561,2364,3181,123,572,31,60620,3969,910,10026,...,127597,2160,640,1,1987,35,3115,1157,1240,1555


##### 1B-6: Pivot Table on the JH historic **recoveries**, grouped / subtotalled by **Country**

In [37]:
jh_hist_countries_recovered = pd.pivot_table(jh_hist_global_recovered, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_recovered = jh_hist_countries_recovered.transpose()
jh_hist_countries_recovered.index = pd.to_datetime(jh_hist_countries_recovered.index)
jh_hist_countries_recovered = jh_hist_countries_recovered.sort_index()

In [38]:
jh_hist_countries_recovered.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-04-20,52272,102601,83636,12285,22647,995,2407853,189688,23177,560492,...,14054,139590,84533,1,168418,2490,251663,2281,88900,35058
2021-04-21,52301,103066,83765,12334,22882,995,2424675,190271,23181,563256,...,14104,142549,84717,1,169685,2490,254199,2309,88956,35065
2021-04-22,52348,103582,83900,12375,22901,995,2449793,191158,23183,565513,...,14195,145464,85007,1,171043,2490,256559,2393,89117,35073


##### 1B-7: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [39]:
jh_hist_uscounties_cases.sort_values('Province_State').head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,4/13/21,4/14/21,4/15/21,4/16/21,4/17/21,4/18/21,4/19/21,4/20/21,4/21/21,4/22/21
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,6734,6740,6748,6750,6760,6763,6763,6773,6793,6819
37,84001075,US,USA,840,1075.0,Lamar,Alabama,US,33.779950,-88.096680,...,1393,1394,1396,1397,1397,1399,1400,1400,1398,1398
38,84001077,US,USA,840,1077.0,Lauderdale,Alabama,US,34.901719,-87.656247,...,9223,9247,9254,9265,9271,9276,9280,9286,9293,9301
39,84001079,US,USA,840,1079.0,Lawrence,Alabama,US,34.520415,-87.310695,...,2953,2953,2953,2959,2962,2964,2963,2969,2976,2977
40,84001081,US,USA,840,1081.0,Lee,Alabama,US,32.601549,-85.351322,...,15541,15552,15567,15589,15614,15631,15638,15642,15670,15694


In [40]:
date_columns = jh_hist_uscounties_cases.iloc[:,11:].columns

In [41]:
jh_hist_usstates_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_cases = jh_hist_usstates_cases.transpose()
jh_hist_usstates_cases.index = pd.to_datetime(jh_hist_usstates_cases.index)
jh_hist_usstates_cases = jh_hist_usstates_cases.sort_index()

In [42]:
jh_hist_usstates_cases.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-04-20,523955,66479,0,855155,333709,3723446,493029,332139,101707,49,...,836563,2859892,393272,22112,3041,648347,388718,149462,652879,57456
2021-04-21,524367,66680,0,855804,334061,3725088,494521,332995,101993,49,...,837833,2864319,393862,22238,3041,649608,390214,149888,653670,57518
2021-04-22,525049,66909,0,856451,334222,3727069,496882,333732,102267,49,...,839841,2867748,394334,22325,3068,650981,391839,150288,654510,57613


##### 1B-8: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [43]:
jh_hist_uscounties_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.transpose()
jh_hist_uscounties_cases.index = pd.to_datetime(jh_hist_uscounties_cases.index)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.sort_index()

In [44]:
jh_hist_uscounties_cases.tail(3)

Province_State Alabama                                                      \
Admin2         Autauga Baldwin Barbour  Bibb Blount Bullock Butler Calhoun   
2021-04-20        6773   20764    2275  2569   6548    1221   2138   14365   
2021-04-21        6793   20787    2284  2569   6556    1221   2140   14375   
2021-04-22        6819   20815    2289  2573   6563    1223   2143   14391   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2021-04-20         3523     1827  ...    2665    618     3109      755   
2021-04-21         3526     1829  ...    2667    618     3110      757   
2021-04-22         3527     1832  ...    2668    618     3112      757   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2021-04-20           4135  3707  2155          0      898    647  
2021-04-21           4144  3711  2154          0      898    647  
2021-04-22           4151  3715  2157          0      898    647  

[3 rows x 3336 columns]

##### 1B-9: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US State**

In [45]:
jh_hist_usstates_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.transpose()
jh_hist_usstates_deaths.index = pd.to_datetime(jh_hist_usstates_deaths.index)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.sort_index()

In [46]:
jh_hist_usstates_deaths.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-04-20,10798,334,0,17193,5706,61122,6211,8020,1603,0,...,12096,49672,2174,242,27,10625,5407,2789,7427,705
2021-04-21,10807,334,0,17199,5708,61193,6215,8027,1603,0,...,12111,49748,2177,243,27,10640,5422,2800,7430,705
2021-04-22,10824,334,0,17221,5711,61285,6225,8039,1604,0,...,12130,49818,2178,243,27,10653,5428,2808,7438,705


##### 1B-10: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US County**

In [47]:
jh_hist_uscounties_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.transpose()
jh_hist_uscounties_deaths.index = pd.to_datetime(jh_hist_uscounties_deaths.index)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.sort_index()

In [48]:
jh_hist_uscounties_deaths.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2021-04-20         106     302      55   61    133      40     68     311   
2021-04-21         107     302      55   61    133      40     68     313   
2021-04-22         107     303      56   62    133      40     68     313   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2021-04-20          122       45  ...      30     11       31        7   
2021-04-21          122       45  ...      30     11       31        7   
2021-04-22          121       45  ...      30     11       31        7   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2021-04-20             37     9    12          0       26      5  
2021-04-21             37     9    12          0       26      5  
2021-04-22             37     9    12          0       26      5  

[3 rows x 3336 columns]

#### 1C: Export the data

##### 1C-1: Export the JH live global data, grouped / subtotalled by **Country**

In [49]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_countries.csv')
jh_live_countries.to_csv(filename, index=True)

##### 1C-2: Export the JH live global data, grouped / subtotalled by **US State**

In [50]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_usstates.csv')
jh_live_usstates.to_csv(filename, index=True)

##### 1C-3: Export the JH live global data, grouped / subtotalled by **US County**

In [51]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_uscounties.csv')
jh_live_uscounties.to_csv(filename, index=True)

##### 1C-4: Export the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [52]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases.csv')
jh_hist_countries_cases.to_csv(filename, index=True)

##### 1C-5: Export the JH historic **deaths**, grouped / subtotalled by **Country**

In [53]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths.csv')
jh_hist_countries_deaths.to_csv(filename, index=True)

##### 1C-6: Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [54]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered.csv')
jh_hist_countries_recovered.to_csv(filename, index=True)

##### 1C-7: Export the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [55]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases.csv')
jh_hist_usstates_cases.to_csv(filename, index=True)

##### 1C-8: Export the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [56]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases.csv')
jh_hist_uscounties_cases.to_csv(filename, index=True)

##### 1C-9: Export the JH historic **deaths**, grouped / subtotalled by **US State**

In [57]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths.csv')
jh_hist_usstates_deaths.to_csv(filename, index=True)

##### 1C-10: Export the JH historic **deaths**, grouped / subtotalled by **US County**

In [58]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=True)

#### 1D: Calculate daily % change for each historic dataset (match numbering sequence, export as well)

##### 1D-4: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [59]:
jh_hist_countries_cases_pct = jh_hist_countries_cases.pct_change()

In [60]:
jh_hist_countries_cases_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-04-20,0.002401,0.001141,0.001561,0.005389,0.005832,0.0,0.010737,0.003194,0.000677,0.003402,...,0.000574,0.013734,0.002794,0.000000,0.006181,0.003225,0.007121,0.010242,0.001100,0.000423
2021-04-21,0.003050,0.001063,0.001517,0.003340,0.009002,0.0,0.009452,0.004931,0.000609,0.004222,...,0.000550,0.019335,0.003737,0.333333,0.005432,0.004286,0.006149,0.007097,0.000846,0.002772
2021-04-22,0.001683,0.001031,0.001573,0.001935,0.006752,0.0,0.009827,0.004185,0.001487,0.004008,...,0.000627,0.019061,0.004386,0.000000,0.007058,0.004267,0.005776,0.010067,0.000768,0.001001


In [61]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct.csv')
jh_hist_countries_cases_pct.to_csv(filename, index=True)

##### 1D-5: Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [62]:
jh_hist_countries_deaths_pct = jh_hist_countries_deaths.pct_change()

In [63]:
jh_hist_countries_deaths_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-04-20,0.001178,0.002556,0.001582,0.0,0.003552,0.0,0.005313,0.006162,0.0,0.003729,...,0.000259,0.025875,0.001567,NaN,0.009870,0.0,0.010174,0.005300,0.000000,0.000644
2021-04-21,0.003138,0.002125,0.002212,0.0,0.008850,0.0,0.004867,0.006379,0.0,0.003816,...,0.000157,0.030168,0.001565,inf,0.010802,0.0,0.005848,0.007909,0.001618,0.000644
2021-04-22,0.001564,0.002545,0.002837,0.0,0.003509,0.0,0.008938,0.006339,0.0,0.002901,...,0.000157,0.036966,0.000000,0.0,0.011196,0.0,0.006137,0.008718,0.001616,0.000000


In [64]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct.csv')
jh_hist_countries_deaths_pct.to_csv(filename, index=True)

##### 1D-6: Daily % change Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [65]:
jh_hist_countries_recovered_pct = jh_hist_countries_recovered.pct_change()

In [66]:
jh_hist_countries_recovered_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-04-20,0.000536,0.004209,0.001461,0.006720,0.002080,0.0,0.008997,0.003550,0.000173,0.005426,...,0.000997,0.024108,0.002681,0.0,0.009543,0.006061,0.009147,0.008846,0.00045,0.000771
2021-04-21,0.000555,0.004532,0.001542,0.003989,0.010377,0.0,0.006986,0.003073,0.000173,0.004931,...,0.003558,0.021198,0.002177,0.0,0.007523,0.000000,0.010077,0.012275,0.00063,0.000200
2021-04-22,0.000899,0.005007,0.001612,0.003324,0.000830,0.0,0.010359,0.004662,0.000086,0.004007,...,0.006452,0.020449,0.003423,0.0,0.008003,0.000000,0.009284,0.036379,0.00181,0.000228


In [67]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct.csv')
jh_hist_countries_recovered_pct.to_csv(filename, index=True)

##### 1D-7: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [68]:
jh_hist_usstates_cases_pct = jh_hist_usstates_cases.pct_change()

In [69]:
jh_hist_usstates_cases_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-04-20,0.002762,0.001642,NaN,0.000822,0.000594,0.000684,0.003997,0.002227,0.001576,0.0,...,0.000863,0.001695,0.000802,0.000000,0.000000,0.001910,0.004647,0.002112,0.001428,0.001359
2021-04-21,0.000786,0.003024,NaN,0.000759,0.001055,0.000441,0.003026,0.002577,0.002812,0.0,...,0.001518,0.001548,0.001500,0.005698,0.000000,0.001945,0.003849,0.002850,0.001212,0.001079
2021-04-22,0.001301,0.003434,NaN,0.000756,0.000482,0.000532,0.004774,0.002213,0.002686,0.0,...,0.002397,0.001197,0.001198,0.003912,0.008879,0.002114,0.004164,0.002669,0.001285,0.001652


In [70]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct.csv')
jh_hist_usstates_cases_pct.to_csv(filename, index=True)

##### 1D-8: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [71]:
jh_hist_uscounties_cases_pct = jh_hist_uscounties_cases.pct_change()

In [72]:
jh_hist_uscounties_cases_pct.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2021-04-20      0.001479  0.001640  0.001761  0.000779  0.002449  0.000820   
2021-04-21      0.002953  0.001108  0.003956  0.000000  0.001222  0.000000   
2021-04-22      0.003827  0.001347  0.002189  0.001557  0.001068  0.001638   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2021-04-20      0.002814  0.000488  0.001137  0.000000  ...  0.000000   
2021-04-21      0.000935  0.000696  0.000852  0.001095  ...  0.000750   
2021-04-22      0.001402  0.001113  0.000284  0.001640  ...  0.000375   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2021-04-20      0.001621  0.001288  0.001326   0.002181  0.001080  0.002326   
2021-04-21      0.000000  0.000322  0.002649   0.002177  0.001079 -0.000464   
2021-04-22      0.000000  0.000643  0.000000   0.001689  0.001078  0.001393   

Province_State                             
Admin2         Unassigned Washakie Weston  
2021-04-20            NaN      0.0    0.0  
2021-04-21            NaN      0.0    0.0  
2021-04-22            NaN      0.0    0.0  

[3 rows x 3336 columns]

In [73]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct.csv')
jh_hist_uscounties_cases_pct.to_csv(filename, index=True)

##### 1D-9: Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [74]:
jh_hist_usstates_deaths_pct = jh_hist_usstates_deaths.pct_change()

In [75]:
jh_hist_usstates_deaths_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-04-20,0.000741,0.0,NaN,0.002332,0.001228,0.001261,0.001613,0.000749,0.000624,NaN,...,0.001242,0.001088,0.003693,0.000000,0.0,0.002832,0.002410,0.001436,0.000809,0.002845
2021-04-21,0.000833,0.0,NaN,0.000349,0.000351,0.001162,0.000644,0.000873,0.000000,NaN,...,0.001240,0.001530,0.001380,0.004132,0.0,0.001412,0.002774,0.003944,0.000404,0.000000
2021-04-22,0.001573,0.0,NaN,0.001279,0.000526,0.001503,0.001609,0.001495,0.000624,NaN,...,0.001569,0.001407,0.000459,0.000000,0.0,0.001222,0.001107,0.002857,0.001077,0.000000


In [76]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct.csv')
jh_hist_usstates_deaths_pct.to_csv(filename, index=True)

##### 1D-10: Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [77]:
jh_hist_uscounties_deaths_pct = jh_hist_uscounties_deaths.pct_change()

In [78]:
jh_hist_uscounties_deaths_pct.tail(3)

Province_State   Alabama                                                      \
Admin2           Autauga   Baldwin   Barbour      Bibb Blount Bullock Butler   
2021-04-20      0.000000  0.000000  0.000000  0.000000    0.0     0.0    0.0   
2021-04-21      0.009434  0.000000  0.000000  0.000000    0.0     0.0    0.0   
2021-04-22      0.000000  0.003311  0.018182  0.016393    0.0     0.0    0.0   

Province_State                                ... Wyoming                  \
Admin2           Calhoun  Chambers  Cherokee  ...    Park Platte Sheridan   
2021-04-20      0.000000  0.000000  0.022727  ...     0.0    0.0      0.0   
2021-04-21      0.006431  0.000000  0.000000  ...     0.0    0.0      0.0   
2021-04-22      0.000000 -0.008197  0.000000  ...     0.0    0.0      0.0   

Province_State                                                             
Admin2         Sublette Sweetwater Teton Uinta Unassigned Washakie Weston  
2021-04-20          0.0        0.0   0.0   0.0        NaN      0.0    0.0  
2021-04-21          0.0        0.0   0.0   0.0        NaN      0.0    0.0  
2021-04-22          0.0        0.0   0.0   0.0        NaN      0.0    0.0  

[3 rows x 3336 columns]

In [79]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct.csv')
jh_hist_uscounties_deaths_pct.to_csv(filename, index=True)

#### 1E: Calculate exponential weighted moving average (3 day) for each historic dataset (match numbering sequence, export as well)

This is useful to see which countries / states / counties are becoming hot spots or which are cooling down.  
Exponential weighted moving averages (EWMA) blend the most recent x days, using a heavier weight for more recent data.  
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html  
Using the EWMA gives the best balance of minimizing erratic daily changes by using a moving avg, while giving more credence to recent data vs a simple moving avg.  

##### 1E-4: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [80]:
jh_hist_countries_cases_pct_ewma = jh_hist_countries_cases_pct.ewm(span=3).mean()

In [81]:
jh_hist_countries_cases_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-04-20,0.002013,0.001058,0.001460,0.003532,0.005542,0.000585,0.009123,0.002837,0.000668,0.003520,...,0.000585,0.014608,0.003177,1.136868e-13,0.006767,0.002916,0.006138,0.009352,0.000908,0.001504
2021-04-21,0.002531,0.001061,0.001488,0.003436,0.007272,0.000293,0.009287,0.003884,0.000638,0.003871,...,0.000567,0.016972,0.003457,1.666667e-01,0.006100,0.003601,0.006144,0.008225,0.000877,0.002138
2021-04-22,0.002107,0.001046,0.001531,0.002686,0.007012,0.000146,0.009557,0.004034,0.001062,0.003939,...,0.000597,0.018016,0.003922,8.333333e-02,0.006579,0.003934,0.005960,0.009146,0.000822,0.001569


In [82]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct_ewma.csv')
jh_hist_countries_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-5: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [83]:
jh_hist_countries_deaths_pct_ewma = jh_hist_countries_deaths_pct.ewm(span=3).mean()

In [84]:
jh_hist_countries_deaths_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-04-20,0.001424,0.002087,0.001459,0.000535,0.003000,0.000139,0.004187,0.005507,2.148652e-06,0.003199,...,0.000184,0.029805,0.001325,NaN,0.009900,2.304940e-71,0.009972,0.006056,0.000452,0.000518
2021-04-21,0.002281,0.002106,0.001835,0.000267,0.005925,0.000070,0.004527,0.005943,1.074326e-06,0.003507,...,0.000171,0.029986,0.001445,NaN,0.010351,1.152470e-71,0.007910,0.006982,0.001035,0.000581
2021-04-22,0.001923,0.002325,0.002336,0.000134,0.004717,0.000035,0.006732,0.006141,5.371631e-07,0.003204,...,0.000164,0.033476,0.000722,0.0,0.010774,5.762350e-72,0.007024,0.007850,0.001325,0.000290


In [85]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct_ewma.csv')
jh_hist_countries_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-6: EWMA of Daily % change Export the JH historic **recovered**, grouped / subtotalled by **Country**

In [86]:
jh_hist_countries_recovered_pct_ewma = jh_hist_countries_recovered_pct.ewm(span=3).mean()

In [87]:
jh_hist_countries_recovered_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-04-20,0.000805,0.004795,0.001425,0.004463,0.002333,0.004839,0.009043,0.004656,0.000321,0.004888,...,0.002122,0.025221,0.002711,0.0,0.008211,0.003811,0.008682,0.010883,0.001043,0.000598
2021-04-21,0.000680,0.004663,0.001484,0.004226,0.006355,0.002419,0.008015,0.003865,0.000247,0.004910,...,0.002840,0.023209,0.002444,0.0,0.007867,0.001905,0.009380,0.011579,0.000836,0.000399
2021-04-22,0.000789,0.004835,0.001548,0.003775,0.003593,0.001210,0.009187,0.004263,0.000166,0.004458,...,0.004646,0.021829,0.002934,0.0,0.007935,0.000953,0.009332,0.023979,0.001323,0.000313


In [88]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct_ewma.csv')
jh_hist_countries_recovered_pct_ewma.to_csv(filename, index=True)

##### 1E-7: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [89]:
jh_hist_usstates_cases_pct_ewma = jh_hist_usstates_cases_pct.ewm(span=3).mean()

In [90]:
jh_hist_usstates_cases_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-04-20,0.001614,0.002708,NaN,0.000802,0.000444,0.000659,0.003160,0.003098,0.002748,1.318329e-121,...,0.002151,0.001424,0.000745,0.002529,0.001252,0.001892,0.003767,0.002175,0.001159,0.001238
2021-04-21,0.001200,0.002866,NaN,0.000781,0.000749,0.000550,0.003093,0.002837,0.002780,6.591646e-122,...,0.001834,0.001486,0.001123,0.004114,0.000626,0.001918,0.003808,0.002513,0.001185,0.001159
2021-04-22,0.001250,0.003150,NaN,0.000768,0.000616,0.000541,0.003934,0.002525,0.002733,3.295823e-122,...,0.002116,0.001341,0.001161,0.004013,0.004752,0.002016,0.003986,0.002591,0.001235,0.001405


In [91]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct_ewma.csv')
jh_hist_usstates_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-8: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [92]:
jh_hist_uscounties_cases_pct_ewma = jh_hist_uscounties_cases_pct.ewm(span=3).mean()

In [93]:
jh_hist_uscounties_cases_pct_ewma.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2021-04-20      0.000930  0.001053  0.001161  0.000846  0.001446  0.000712   
2021-04-21      0.001942  0.001081  0.002559  0.000423  0.001334  0.000356   
2021-04-22      0.002884  0.001214  0.002374  0.000990  0.001201  0.000997   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2021-04-20      0.001560  0.000428  0.000795  0.000080  ...  0.000238   
2021-04-21      0.001248  0.000562  0.000823  0.000587  ...  0.000494   
2021-04-22      0.001325  0.000838  0.000553  0.001114  ...  0.000435   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2021-04-20      0.000903  0.001165  0.000959   0.001559  0.001246  0.000615   
2021-04-21      0.000451  0.000744  0.001804   0.001868  0.001163  0.000075   
2021-04-22      0.000226  0.000693  0.000902   0.001778  0.001120  0.000734   

Province_State                                 
Admin2         Unassigned  Washakie    Weston  
2021-04-20           -1.0  0.000006  0.000446  
2021-04-21           -1.0  0.000003  0.000223  
2021-04-22           -1.0  0.000001  0.000111  

[3 rows x 3336 columns]

In [94]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct_ewma.csv')
jh_hist_uscounties_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-9: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [95]:
jh_hist_usstates_deaths_pct_ewma = jh_hist_usstates_deaths_pct.ewm(span=3).mean()

In [96]:
jh_hist_usstates_deaths_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-04-20,0.000698,0.001001,NaN,0.001248,0.000933,0.000935,0.001199,0.001021,0.000647,NaN,...,0.001329,0.000897,0.002120,0.000611,0.009615,0.002140,0.002011,0.001189,0.000538,0.001434
2021-04-21,0.000766,0.000501,NaN,0.000798,0.000642,0.001048,0.000922,0.000947,0.000323,NaN,...,0.001284,0.001214,0.001750,0.002371,0.004808,0.001776,0.002393,0.002567,0.000471,0.000717
2021-04-22,0.001169,0.000250,NaN,0.001039,0.000584,0.001276,0.001265,0.001221,0.000474,NaN,...,0.001427,0.001310,0.001105,0.001186,0.002404,0.001499,0.001750,0.002712,0.000774,0.000358


In [97]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct_ewma.csv')
jh_hist_usstates_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-10: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [98]:
jh_hist_uscounties_deaths_pct_ewma = jh_hist_uscounties_deaths_pct.ewm(span=3).mean()

In [99]:
jh_hist_uscounties_deaths_pct_ewma.tail(3)

Province_State   Alabama                                              \
Admin2           Autauga   Baldwin       Barbour      Bibb    Blount   
2021-04-20      0.001822  0.000104  8.830911e-09  0.002186 -0.000577   
2021-04-21      0.005628  0.000052  4.415455e-09  0.001093 -0.000289   
2021-04-22      0.002814  0.001682  9.090911e-03  0.008743 -0.000144   

Province_State                                                                \
Admin2               Bullock    Butler       Calhoun      Chambers  Cherokee   
2021-04-20      1.222661e-08  0.001051  8.203937e-07  1.332028e-06  0.011375   
2021-04-21      6.113303e-09  0.000526  3.215844e-03  6.660140e-07  0.005687   
2021-04-22      3.056652e-09  0.000263  1.607922e-03 -4.098028e-03  0.002844   

Province_State  ...       Wyoming                                            \
Admin2          ...          Park        Platte      Sheridan      Sublette   
2021-04-20      ...  6.423760e-11  2.607380e-27  4.851625e-13  2.131363e-33   
2021-04-21      ...  3.211880e-11  1.303690e-27  2.425813e-13  1.065681e-33   
2021-04-22      ...  1.605940e-11  6.518449e-28  1.212906e-13  5.328407e-34   

Province_State                                                       \
Admin2            Sweetwater         Teton         Uinta Unassigned   
2021-04-20      1.324548e-08  6.803710e-21  3.457604e-28  -0.501961   
2021-04-21      6.622738e-09  3.401855e-21  1.728802e-28  -0.501961   
2021-04-22      3.311369e-09  1.700928e-21  8.644010e-29  -0.501961   

Province_State                              
Admin2              Washakie        Weston  
2021-04-20      1.324251e-25  2.812414e-24  
2021-04-21      6.621257e-26  1.406207e-24  
2021-04-22      3.310629e-26  7.031035e-25  

[3 rows x 3336 columns]

In [100]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct_ewma.csv')
jh_hist_uscounties_deaths_pct_ewma.to_csv(filename, index=True)

### Dataset 2: US States - Covid Tracking Project

The Covid Tracking Project stores data in both API (JSON) and CSV formats.  We'll be loading the CSV files into Pandas here.  
https://covidtracking.com/api/  

States current - /api/v1/states/current.csv | [CSV](https://covidtracking.com/api/v1/states/current.csv)  
States historic - /api/v1/states/daily.csv | [CSV](https://covidtracking.com/api/v1/states/daily.csv)  
States info - /api/v1/states/info.csv | [CSV](https://covidtracking.com/api/v1/states/info.csv)   

#### 2A: Pull the Data from CT's API, immediately export as 'orig'

In [101]:
ct_live_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/current.csv")
ct_live_usstates.head()

,date,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20210307,AK,56886,NaN,NaN,NaN,totalTestsViral,1731628,33.0,1293.0,...,NaN,0,0,dc4bccd4bb885349d7e94d6fed058e285d4be164,0,0,0,0,0,NaN
1,20210307,AL,499819,107742.0,1931711.0,NaN,totalTestsPeopleViral,2323788,494.0,45976.0,...,NaN,-1,0,997207b430824ea40b8eb8506c19a93e07bc972e,0,0,0,0,0,NaN
2,20210307,AR,324818,69092.0,2480716.0,NaN,totalTestsViral,2736442,335.0,14926.0,...,NaN,22,11,50921aeefba3e30d31623aa495b47fb2ecc72fae,0,0,0,0,0,NaN
3,20210307,AS,0,NaN,2140.0,NaN,totalTestsViral,2140,NaN,NaN,...,NaN,0,0,96d23f888c995b9a7f3b4b864de6414f45c728ff,0,0,0,0,0,NaN
4,20210307,AZ,826454,56519.0,3073010.0,NaN,totalTestsViral,7908105,963.0,57907.0,...,NaN,5,44,0437a7a96f4471666f775e63e86923eb5cbd8cdf,0,0,0,0,0,NaN


In [102]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_live_usstates_orig.csv')
ct_live_usstates.to_csv(filename, index=False)

In [103]:
ct_hist_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/daily.csv")
ct_hist_usstates.head()

,date,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20210307,AK,56886.0,NaN,NaN,NaN,totalTestsViral,1731628.0,33.0,1293.0,...,NaN,0,0,dc4bccd4bb885349d7e94d6fed058e285d4be164,0,0,0,0,0,NaN
1,20210307,AL,499819.0,107742.0,1931711.0,NaN,totalTestsPeopleViral,2323788.0,494.0,45976.0,...,NaN,-1,0,997207b430824ea40b8eb8506c19a93e07bc972e,0,0,0,0,0,NaN
2,20210307,AR,324818.0,69092.0,2480716.0,NaN,totalTestsViral,2736442.0,335.0,14926.0,...,NaN,22,11,50921aeefba3e30d31623aa495b47fb2ecc72fae,0,0,0,0,0,NaN
3,20210307,AS,0.0,NaN,2140.0,NaN,totalTestsViral,2140.0,NaN,NaN,...,NaN,0,0,96d23f888c995b9a7f3b4b864de6414f45c728ff,0,0,0,0,0,NaN
4,20210307,AZ,826454.0,56519.0,3073010.0,NaN,totalTestsViral,7908105.0,963.0,57907.0,...,NaN,5,44,0437a7a96f4471666f775e63e86923eb5cbd8cdf,0,0,0,0,0,NaN


In [104]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_hist_usstates_orig.csv')
ct_hist_usstates.to_csv(filename, index=False)

#### 2B: Change the dates to datetime format, make them the index for both datasets

In [105]:
ct_live_usstates['date'] = ct_live_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_live_usstates['date'] = pd.to_datetime(ct_live_usstates['date'])
ct_live_usstates.set_index('date', inplace=True)
ct_live_usstates.head()

,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2021-03-07,AK,56886,NaN,NaN,NaN,totalTestsViral,1731628,33.0,1293.0,NaN,...,NaN,0,0,dc4bccd4bb885349d7e94d6fed058e285d4be164,0,0,0,0,0,NaN
2021-03-07,AL,499819,107742.0,1931711.0,NaN,totalTestsPeopleViral,2323788,494.0,45976.0,NaN,...,NaN,-1,0,997207b430824ea40b8eb8506c19a93e07bc972e,0,0,0,0,0,NaN
2021-03-07,AR,324818,69092.0,2480716.0,NaN,totalTestsViral,2736442,335.0,14926.0,141.0,...,NaN,22,11,50921aeefba3e30d31623aa495b47fb2ecc72fae,0,0,0,0,0,NaN
2021-03-07,AS,0,NaN,2140.0,NaN,totalTestsViral,2140,NaN,NaN,NaN,...,NaN,0,0,96d23f888c995b9a7f3b4b864de6414f45c728ff,0,0,0,0,0,NaN
2021-03-07,AZ,826454,56519.0,3073010.0,NaN,totalTestsViral,7908105,963.0,57907.0,273.0,...,NaN,5,44,0437a7a96f4471666f775e63e86923eb5cbd8cdf,0,0,0,0,0,NaN


In [106]:
ct_hist_usstates['date'] = ct_hist_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_hist_usstates['date'] = pd.to_datetime(ct_hist_usstates['date'])
ct_hist_usstates.set_index('date', inplace=True)
ct_hist_usstates.head()

,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2021-03-07,AK,56886.0,NaN,NaN,NaN,totalTestsViral,1731628.0,33.0,1293.0,NaN,...,NaN,0,0,dc4bccd4bb885349d7e94d6fed058e285d4be164,0,0,0,0,0,NaN
2021-03-07,AL,499819.0,107742.0,1931711.0,NaN,totalTestsPeopleViral,2323788.0,494.0,45976.0,NaN,...,NaN,-1,0,997207b430824ea40b8eb8506c19a93e07bc972e,0,0,0,0,0,NaN
2021-03-07,AR,324818.0,69092.0,2480716.0,NaN,totalTestsViral,2736442.0,335.0,14926.0,141.0,...,NaN,22,11,50921aeefba3e30d31623aa495b47fb2ecc72fae,0,0,0,0,0,NaN
2021-03-07,AS,0.0,NaN,2140.0,NaN,totalTestsViral,2140.0,NaN,NaN,NaN,...,NaN,0,0,96d23f888c995b9a7f3b4b864de6414f45c728ff,0,0,0,0,0,NaN
2021-03-07,AZ,826454.0,56519.0,3073010.0,NaN,totalTestsViral,7908105.0,963.0,57907.0,273.0,...,NaN,5,44,0437a7a96f4471666f775e63e86923eb5cbd8cdf,0,0,0,0,0,NaN
